## Homework

Исходные данные - Yandex Cup 2022 RecSys:
- Описание соревнования - https://contest.yandex.ru/yacup/contest/41618/problems/
- Данные - https://disk.yandex.ru/d/SI1aAooPn9i8TA
- Описание данных - в архиве likes_data.zip три файла:
  - train - обучающий набор данных. Каждая строка - последовательность id треков, которые лайкнул один пользователь. Гарантируется, что лайки даны в той последовательности, в которой их ставил пользователь.
  - test - набор тестовых данных. Имеет точно такой же формат, но в каждой строке не хватает последнего лайка, который надо предсказать.
  - track_artists.csv - информация о исполнителях треков. Гарантируется, что у каждого трека есть ровно один исполнитель. Для треков, у которых фактически несколько исполнителей, мы оставили того, который считается основным исполнителем трека.
- Описание сабмита - в качестве решения необходимо отправить файл, в котором для каждого пользователя из test в отдельной строке будет не более 100 треков, разделенных пробелом. Гарантируется, что у каждого пользователя будет только 1 лайк в тесте
- Метрика - MRR@100

In [11]:
import pandas as pd
import numpy as np

import tqdm

## 1. Скачивание и распаковка датасета

In [12]:
!mkdir dataset

In [13]:
import requests
from urllib.parse import urlencode

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/SI1aAooPn9i8TA'

# Получаем загрузочную ссылку
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']

# Загружаем файл и сохраняем его
download_response = requests.get(download_url)
with open('dataset/data.zip', 'wb') as f:   # Здесь укажите нужный путь к файлу
    f.write(download_response.content)

In [14]:
import zipfile

with zipfile.ZipFile("dataset/data.zip", mode="r") as archive:
    archive.extractall("dataset")

In [15]:
with zipfile.ZipFile("dataset/likes/likes_data.zip", mode="r") as archive:
    archive.extractall("dataset")

In [16]:
!del "dataset\data.zip"
!rd /s /q "dataset\likes"

## 2. Преобразование данных

In [17]:
likes = []

with open('dataset/train', 'r') as f:
    for i, line in enumerate(tqdm.tqdm(f.readlines())):
        track_ids = [int(n) for n in line.split()]
        user_likes = np.zeros((len(track_ids), 2), dtype=int)
        user_likes[:, 0] = i
        # в позиции 0 - самый свежий лайк
        user_likes[:, 1] = track_ids[::-1]
        likes.append(user_likes)

all_likes = np.vstack(likes)

100%|█████████████████████████████████████████████████████████████████████| 1160084/1160084 [00:28<00:00, 41217.59it/s]


In [18]:
likes_df = pd.DataFrame(all_likes, columns = ['user_id', 'track_id'])
print(likes_df.shape)
likes_df.head()

(94188634, 2)


,user_id,track_id
0,0,388242
1,0,278503
2,0,102795
3,0,470957
4,0,159637


## 3. Валидация

In [29]:
from sklearn.model_selection import KFold

class UsersKFoldPOut():
    def __init__(self, n_folds: int, p: int, random_seed: int=23):
        self.n_folds = n_folds
        self.p = p
        self.random_seed = random_seed
    
    def split(self, df: pd.DataFrame):
        df = df.copy()
        users = df['user_id'].unique()
        # Порядкоые номера лайков
        like_numbers = df.groupby('user_id').cumcount()
        # Разбиваем user'ов на фолды с необходимыми параметрами
        user_kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=self.random_seed)
        for train_users, test_users in user_kfold.split(users):
            # Получим маски с учетом пользователей и порядковых номеров лайков (для test)
            train_mask = df['user_id'].isin(train_users)
            test_mask = df['user_id'].isin(test_users) & (like_numbers < self.p)
            yield train_mask, test_mask

In [30]:
n_folds = 3
p = 3

cv = UsersKFoldPOut(n_folds=n_folds, p=p)

for i, (train_mask, test_mask) in enumerate(cv.split(likes_df)):
    train_fold = likes_df[train_mask]
    test_fold = likes_df[test_mask]
    assert len(set(train_fold['user_id'].unique()).intersection(test_fold['user_id'].unique())) == 0, 'Found common users in train and test'
    assert test_fold.groupby('user_id').count().values.max() <= p, 'More than p track_ids in test fold'
    # Проверим, что только последние лайки попали в тест
    test_first_user = test_fold.iloc[0]['user_id']
    test_fold_user_tracks = test_fold[test_fold['user_id'] == test_first_user]['track_id'].values
    all_user_tracks = likes_df[likes_df['user_id'] == test_first_user]['track_id'].values
    assert np.all(test_fold_user_tracks == all_user_tracks[:p]), 'Wrong user track_ids in test'